# E-Commerce Shipping Data

Una empresa internacional de comercio electrónico busca extraer información clave de su base de datos de clientes. Quiere utilizar técnicas de aprendizaje automático avanzadas para estudiar a sus clientes. La empresa vende productos electrónicos.

**Contenido**

El conjunto de datos utilizado para la construcción del modelo contenía 10 999 observaciones de 12 variables.
Los datos contienen la siguiente información:

ID: Número de identificación de los clientes.

Bloque de almacén: La empresa cuenta con un gran almacén dividido en bloques A, B, C, D y E.

Modo de envío: La empresa envía los productos por diferentes medios, como barco, avión y carretera.

Llamadas de atención al cliente: Número de llamadas realizadas para solicitar información sobre el envío.

Calificación del cliente: La empresa ha calificado a cada cliente. 1 es la puntuación más baja (la peor), 5 es la más alta (la mejor).

Costo del producto: Costo del producto en dólares estadounidenses.

Compras anteriores: Número de compras anteriores. Importancia del producto: La empresa ha categorizado el producto según diversos parámetros, como bajo, medio y alto.

Género: Masculino y Femenino.

Descuento ofrecido: Descuento ofrecido en ese producto específico.

Peso en gramos: Peso en gramos.

Entrega a tiempo: Es la variable objetivo, donde 1 indica que el producto no se entregó a tiempo y 0 indica que se entregó a tiempo.

**Preguntas de análisis**

¿Cuál es el porcentaje de productos que no llegaron a tiempo?

¿Qué modo de envío tiene mayor tasa de entregas tardías?

¿Cuál es la calificación promedio dada por los clientes según el género?

¿Hay relación entre la cantidad de llamadas al servicio al cliente y las entregas tardías?

¿Cuál es el peso promedio de los productos entregados a tiempo vs. no a tiempo?

¿Cuál es el costo promedio del producto según su nivel de importancia?

¿Qué bloque de almacén tiene más productos que no llegaron a tiempo?

¿Cuál es el descuento promedio ofrecido por género?

¿Qué nivel de importancia del producto tiene mayor proporción de entregas tardías?

¿Cuántas compras previas tienen en promedio los clientes que recibieron productos a tiempo vs. no a tiempo?

In [1]:
import pandas as pd
import sqlite3

# Cargar el archivo CSV
df = pd.read_csv("Train.csv")

# Crear conexión SQLite en memoria
conn = sqlite3.connect(":memory:")

# Cargar el DataFrame como tabla
df.to_sql("shipping", conn, index=False, if_exists="replace")

10999

In [4]:
#¿Cuál es el porcentaje de productos que no llegaron a tiempo?

query1 = """
SELECT
  ROUND(SUM(CASE WHEN "Reached on time" = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS Late_Delivery_Percentage
FROM shipping
"""
pd.read_sql_query(query1, conn)


,Late_Delivery_Percentage
0,0.0


In [8]:
#¿Qué modo de envío tiene mayor tasa de entregas tardías?

query2 = """
SELECT
  Mode_of_Shipment,
  COUNT(*) AS Total,
  SUM(CASE WHEN "Reached.on.Time_Y.N" = 1 THEN 1 ELSE 0 END) AS Late_Deliveries,
  ROUND(SUM(CASE WHEN "Reached.on.Time_Y.N" = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS Late_Percentage
FROM shipping
GROUP BY Mode_of_Shipment
ORDER BY Late_Percentage DESC
"""
pd.read_sql_query(query2, conn)



,Mode_of_Shipment,Total,Late_Deliveries,Late_Percentage
0,Flight,1777,1069,60.16
1,Ship,7462,4459,59.76
2,Road,1760,1035,58.81


In [9]:
#¿Cuál es la calificación promedio dada por los clientes según el género?
query3 = """
SELECT
  Gender,
  ROUND(AVG(Customer_rating), 2) AS Avg_Rating
FROM shipping
GROUP BY Gender
"""
pd.read_sql_query(query3, conn)


,Gender,Avg_Rating
0,F,2.99
1,M,2.99


In [10]:
#¿Hay relación entre la cantidad de llamadas al servicio al cliente y las entregas tardías?

query4 = """
SELECT
  Customer_care_calls,
  ROUND(AVG("Reached.on.Time_Y.N"), 2) AS Late_Delivery_Rate
FROM shipping
GROUP BY Customer_care_calls
ORDER BY Customer_care_calls
"""
pd.read_sql_query(query4, conn)

,Customer_care_calls,Late_Delivery_Rate
0,2,0.65
1,3,0.63
2,4,0.60
3,5,0.58
4,6,0.52
5,7,0.52


In [11]:
#¿Cuál es el peso promedio de los productos entregados a tiempo vs. no a tiempo?

query5 = """
SELECT
  "Reached.on.Time_Y.N" AS Late,
  ROUND(AVG(Weight_in_gms), 2) AS Avg_Weight
FROM shipping
GROUP BY "Reached.on.Time_Y.N"
"""
pd.read_sql_query(query5, conn)


,Late,Avg_Weight
0,0,4168.67
1,1,3272.64


In [12]:
#¿Cuál es el costo promedio del producto según su nivel de importancia?

query6 = """
SELECT
  Product_importance,
  ROUND(AVG(Cost_of_the_Product), 2) AS Avg_Cost
FROM shipping
GROUP BY Product_importance
"""
pd.read_sql_query(query6, conn)


,Product_importance,Avg_Cost
0,high,203.87
1,low,212.06
2,medium,209.38


In [13]:
#¿Qué bloque de almacén tiene más productos que no llegaron a tiempo?

query7 = """
SELECT
  Warehouse_block,
  COUNT(*) AS Total_Orders,
  SUM(CASE WHEN "Reached.on.Time_Y.N" = 1 THEN 1 ELSE 0 END) AS Late_Deliveries
FROM shipping
GROUP BY Warehouse_block
ORDER BY Late_Deliveries DESC
"""
pd.read_sql_query(query7, conn)


,Warehouse_block,Total_Orders,Late_Deliveries
0,F,3666,2194
1,B,1833,1104
2,D,1834,1096
3,C,1833,1094
4,A,1833,1075


In [14]:
#¿Cuál es el descuento promedio ofrecido por género?

query8 = """
SELECT
  Gender,
  ROUND(AVG(Discount_offered), 2) AS Avg_Discount
FROM shipping
GROUP BY Gender
"""
pd.read_sql_query(query8, conn)


,Gender,Avg_Discount
0,F,13.56
1,M,13.18


In [15]:
#¿Qué nivel de importancia del producto tiene mayor proporción de entregas tardías?

query9 = """
SELECT
  Product_importance,
  COUNT(*) AS Total,
  SUM(CASE WHEN "Reached.on.Time_Y.N" = 1 THEN 1 ELSE 0 END) AS Late_Deliveries,
  ROUND(SUM(CASE WHEN "Reached.on.Time_Y.N" = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS Late_Percentage
FROM shipping
GROUP BY Product_importance
ORDER BY Late_Percentage DESC
"""
pd.read_sql_query(query9, conn)


,Product_importance,Total,Late_Deliveries,Late_Percentage
0,high,948,616,64.98
1,low,5297,3140,59.28
2,medium,4754,2807,59.05


In [16]:
#¿Cuántas compras previas tienen en promedio los clientes que recibieron productos a tiempo vs. no a tiempo?

query10 = """
SELECT
  "Reached.on.Time_Y.N" AS Late,
  ROUND(AVG(Prior_purchases), 2) AS Avg_Prior_Purchases
FROM shipping
GROUP BY "Reached.on.Time_Y.N"
"""
pd.read_sql_query(query10, conn)


,Late,Avg_Prior_Purchases
0,0,3.67
1,1,3.50
